# 4.1. 리소스 클린업

# 1. 환경 설정

In [48]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
%store -r repository_name
%store -r code_build_project_name
%store -r sm_pipeline_name
%store -r code_pipeline_name
%store -r bucket
%store -r endpoint_name

In [50]:
print("endpoint_name: ", endpoint_name)

endpoint_name:  ncf-codepipeline-endpoint


# 2. 코드 리파지토리 삭제

In [51]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit delete-repository --repository-name $repository_name 


# 3. 코드 빌드 프로젝트 삭제

In [52]:
%%sh -s {code_build_project_name}
code_build_project_name=$1
aws codebuild delete-project --name $code_build_project_name

# 4. 코드 파이프라인 삭제

In [53]:
%%sh -s {code_pipeline_name}
pipeline_name=$1
aws codepipeline delete-pipeline --name $pipeline_name

# 5. 엔드포인트, 엔드포인트 컨피그, 세이즈 메이커 모델 삭제

In [54]:
import boto3

sm_client = boto3.client('sagemaker')
from inference_utils import delete_endpoint
delete_endpoint(client = sm_client, 
                endpoint_name = endpoint_name
               )



#### Start


ClientError: An error occurred (ValidationException) when calling the DescribeEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:057716757052:endpoint/ncf-codepipeline-endpoint".

# 5. SageMaker Model Building Pipeline 삭제




## 5.1 파이프라인 삭제

- 파이프라인 이름 검색

sm_pipeline_name 의 변수 이름으로 시작되는 파이프라인 정보 검색

In [25]:
import boto3

client = boto3.client('sagemaker')

response = client.list_pipelines(
    PipelineNamePrefix= sm_pipeline_name,
    SortBy='Name',
    SortOrder='Descending',
    MaxResults=1
)

response

{'PipelineSummaries': [{'PipelineArn': 'arn:aws:sagemaker:us-east-1:057716757052:pipeline/ncf-sm-pipeline',
   'PipelineName': 'ncf-sm-pipeline',
   'PipelineDisplayName': 'ncf-sm-pipeline',
   'RoleArn': 'arn:aws:iam::057716757052:role/codebuild-gsmoon',
   'CreationTime': datetime.datetime(2022, 9, 1, 14, 29, 59, 513000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2022, 9, 1, 14, 41, 48, 560000, tzinfo=tzlocal())}],
 'ResponseMetadata': {'RequestId': 'd5e88b15-4ebd-44f8-8de2-927ce3fd2380',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd5e88b15-4ebd-44f8-8de2-927ce3fd2380',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '307',
   'date': 'Sun, 13 Nov 2022 11:02:24 GMT'},
  'RetryAttempts': 0}}

In [ ]:
response = client.delete_pipeline(
    PipelineName=sm_pipeline_name,
)
response

## 5.2. 파이프라인 관련 S3 내 파일 삭제

In [ ]:
# ! aws s3 ls {bucket}/{sm_pipeline_name} --recursive
! aws s3 rm s3://{bucket}/{sm_pipeline_name} --recursive

# 6. 모델 레지스트리 삭제

- 자세한 것은 개발자 가이드 참고 하세요. [Boto3 SageMaker](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.delete_model_package)

## 6.1. 전체 model package group 검색
- NameContains 의 이름 있는 것만 검색

In [ ]:
response = client.list_model_package_groups(
    NameContains = 'mimick',
    SortBy='CreationTime',
    SortOrder='Ascending'
)
response

## 6.2. 기술된 model package group 안애서 등록된 model package 만 검색

In [ ]:
model_package_group_name = sm_pipeline_name

In [ ]:
response = client.list_model_packages(
    ModelPackageGroupName=model_package_group_name , 
    SortBy='CreationTime',
    SortOrder='Ascending'
)
response

## 6.3. model package 삭제

In [ ]:
for e in response['ModelPackageSummaryList']:
    print(e)
    ModelPackageArn = e['ModelPackageArn']
    print(f"########## Delete: {ModelPackageArn}")
    response = client.delete_model_package(
        ModelPackageName=ModelPackageArn
    )    
    print(response)

## 6.4. 기술된 model package group 삭제

In [ ]:
response = client.delete_model_package_group(
    ModelPackageGroupName= model_package_group_name
)
response